In [12]:
# from google.colab import files
# from google.colab import drive
import os
import io
import pandas as pd
import missingno as msno
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import chisquare
import numpy as np

from datetime import datetime
# drive.mount('/content/drive') #/MyDrive/Colab Notebooks/데이터/기상데이터

In [4]:
os.getcwd()

'/home/jupyter/code'

In [13]:
cloud = pd.read_csv('../data/cloud.csv', index_col = 0)
lon_lat = pd.read_csv('../data//mapping_0713.csv', index_col = 0)

In [63]:
cloud

,a.tma,a.stn_id,a.avg_tca,a.min_ch
1,2012-01-01 00:00:00.0,90,4.4,NaN
2,2012-01-01 00:00:00.0,95,6.5,NaN
3,2012-01-01 00:00:00.0,98,NaN,NaN
4,2012-01-01 00:00:00.0,100,7.4,NaN
5,2012-01-01 00:00:00.0,101,6.6,NaN
...,...,...,...,...
373990,2022-11-09 00:00:00.0,277,3.1,NaN
373991,2022-11-09 00:00:00.0,284,2.9,NaN
373992,2022-11-09 00:00:00.0,285,3.6,NaN
373993,2022-11-09 00:00:00.0,288,2.3,NaN


In [14]:
cloud = pd.read_csv('../data/cloud.csv', index_col = 0)
lon_lat = pd.read_csv('../data//mapping_0713.csv', index_col = 0)

cloud = cloud.rename(columns={'a.tma' : 'tma', 'a.stn_id' : 'stn_id', 'a.avg_tca' : 'ca', 'a.min_ch' : 'ch'})

# cloud = pd.merge(cloud, lon_lat, on = ['stn_id'])

cloud['date'] = cloud['tma'].map(lambda date_cloud: date_cloud[:10])
cloud['time'] = cloud['tma'].map(lambda time_cloud: time_cloud[11:-2])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')
cloud['time'] = pd.to_datetime(cloud['time'], format='%H:%M:%S')

# del cloud['name']
# del cloud['address']

cloud_ds = cloud.groupby(['date', 'stn_id'], as_index=False).median()

cloud = cloud_ds
cloud = pd.merge(cloud, lon_lat, on = ['stn_id'])

del cloud['name']
del cloud['address']

In [37]:
cloud

,date,stn_id,ca,ch,lat,long,add
0,2012-01-01,90,4.4,NaN,38.250875,128.564715,강원
1,2012-01-02,90,3.5,NaN,38.250875,128.564715,강원
2,2012-01-03,90,3.3,NaN,38.250875,128.564715,강원
3,2012-01-04,90,1.3,NaN,38.250875,128.564715,강원
4,2012-01-05,90,0.0,NaN,38.250875,128.564715,강원
...,...,...,...,...,...,...,...
373629,2022-11-05,239,0.0,NaN,36.485220,127.244380,충남
373630,2022-11-06,239,2.6,NaN,36.485220,127.244380,충남
373631,2022-11-07,239,3.1,NaN,36.485220,127.244380,충남
373632,2022-11-08,239,3.6,NaN,36.485220,127.244380,충남


# -------------------------------------------  
# Linear Interpolation

In [15]:
days = pd.DataFrame({'date' : pd.date_range(start="2012-02-01",end="2022-10-31")})

In [16]:
cloud['date'] = pd.to_datetime(cloud['date'])
cloud['date'] = pd.to_datetime(cloud['date'], format='%Y-%m-%d')

In [17]:
cloud_Gangwon = cloud[cloud['add']=='강원']
cloud_Gyeonggi = cloud[cloud['add']=='경기']
cloud_Gyeongnam = cloud[cloud['add']=='경남']
cloud_Gyeongbuk = cloud[cloud['add']=='경북']
cloud_Gwangju = cloud[cloud['add']=='광주']
cloud_Daegu = cloud[cloud['add']=='대구']
cloud_Daejeon = cloud[cloud['add']=='대전']
cloud_Busan = cloud[cloud['add']=='부산']
cloud_Seoul = cloud[cloud['add']=='서울']
cloud_Sejong = cloud[cloud['add']=='세종']
cloud_Ulsan = cloud[cloud['add']=='울산']
cloud_Incheon = cloud[cloud['add']=='인천']
cloud_Jeonnam = cloud[cloud['add']=='전남']
cloud_Jeonbuk = cloud[cloud['add']=='전북']
cloud_Jeju = cloud[cloud['add']=='제주']
cloud_Chungnam = cloud[cloud['add']=='충남']
cloud_Chungbuk = cloud[cloud['add']=='충북']

## 강원

In [18]:
cloud_Gangwon_outer = pd.merge(cloud_Gangwon, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Gangwon_outer['ca'] = cloud_Gangwon_outer['ca'].interpolate(method = 'values')
cloud_Gangwon_outer['ch'] = cloud_Gangwon_outer['ch'].interpolate(method = 'values')

cloud_Gangwon_outer['add'] = '강원'
del cloud_Gangwon_outer['stn_id']
del cloud_Gangwon_outer['lat']
del cloud_Gangwon_outer['long']
del cloud_Gangwon_outer['ch']

In [19]:
len(set(cloud_Gangwon_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 경기

In [20]:
cloud_Gyeonggi_outer = pd.merge(cloud_Gyeonggi, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Gyeonggi_outer['ca'] = cloud_Gyeonggi_outer['ca'].interpolate(method = 'values')
cloud_Gyeonggi_outer['ch'] = cloud_Gyeonggi_outer['ch'].interpolate(method = 'values')

cloud_Gyeonggi_outer['add'] = '경기'
del cloud_Gyeonggi_outer['stn_id']
del cloud_Gyeonggi_outer['lat']
del cloud_Gyeonggi_outer['long']
del cloud_Gyeonggi_outer['ch']

In [21]:
len(set(cloud_Gyeonggi_outer['date']))

3966

In [67]:
cloud_Gyeonggi_outer.isna().sum()

date    0
ca      2
add     0
dtype: int64

## 경남

In [22]:
cloud_Gyeongnam_outer = pd.merge(cloud_Gyeongnam, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Gyeongnam_outer['ca'] = cloud_Gyeongnam_outer['ca'].interpolate(method = 'values')
cloud_Gyeongnam_outer['ch'] = cloud_Gyeongnam_outer['ch'].interpolate(method = 'values')

cloud_Gyeongnam_outer['add'] = '경남'
del cloud_Gyeongnam_outer['stn_id']
del cloud_Gyeongnam_outer['lat']
del cloud_Gyeongnam_outer['long']
del cloud_Gyeongnam_outer['ch']

In [23]:
len(set(cloud_Gyeongnam_outer['date']))

3966

In [68]:
cloud_Gyeongnam_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 경북

In [24]:
cloud_Gyeongbuk_outer = pd.merge(cloud_Gyeongbuk, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Gyeongbuk_outer['ca'] = cloud_Gyeongbuk_outer['ca'].interpolate(method = 'values')
cloud_Gyeongbuk_outer['ch'] = cloud_Gyeongbuk_outer['ch'].interpolate(method = 'values')

cloud_Gyeongbuk_outer['add'] = '경북'
del cloud_Gyeongbuk_outer['stn_id']
del cloud_Gyeongbuk_outer['lat']
del cloud_Gyeongbuk_outer['long']
del cloud_Gyeongbuk_outer['ch']

In [25]:
len(set(cloud_Gyeongbuk_outer['date']))

3966

In [69]:
cloud_Gyeongbuk_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 광주

In [26]:
cloud_Gwangju_outer = pd.merge(cloud_Gwangju, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Gwangju_outer['ca'] = cloud_Gwangju_outer['ca'].interpolate(method = 'values')
cloud_Gwangju_outer['ch'] = cloud_Gwangju_outer['ch'].interpolate(method = 'values')

cloud_Gwangju_outer['add'] = '광주'
del cloud_Gwangju_outer['stn_id']
del cloud_Gwangju_outer['lat']
del cloud_Gwangju_outer['long']
del cloud_Gwangju_outer['ch']


In [27]:
len(set(cloud_Gwangju_outer['date']))

3966

In [70]:
cloud_Gwangju_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 대구

In [28]:
cloud_Daegu_outer = pd.merge(cloud_Daegu, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Daegu_outer['ca'] = cloud_Daegu_outer['ca'].interpolate(method = 'values')
cloud_Daegu_outer['ch'] = cloud_Daegu_outer['ch'].interpolate(method = 'values')

cloud_Daegu_outer['add'] = '대구'
del cloud_Daegu_outer['stn_id']
del cloud_Daegu_outer['lat']
del cloud_Daegu_outer['long']
del cloud_Daegu_outer['ch']

In [29]:
len(set(cloud_Daegu_outer['date']))

3966

In [71]:
cloud_Daegu_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 대전

In [30]:
cloud_Daejeon_outer = pd.merge(cloud_Daejeon, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Daejeon_outer['ca'] = cloud_Daejeon_outer['ca'].interpolate(method = 'values')
cloud_Daejeon_outer['ch'] = cloud_Daejeon_outer['ch'].interpolate(method = 'values')

cloud_Daejeon_outer['add'] = '대전'
del cloud_Daejeon_outer['stn_id']
del cloud_Daejeon_outer['lat']
del cloud_Daejeon_outer['long']
del cloud_Daejeon_outer['ch']

In [31]:
len(set(cloud_Daejeon_outer['date']))

3966

In [72]:
cloud_Daejeon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 부산

In [32]:
cloud_Busan_outer = pd.merge(cloud_Busan, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Busan_outer['ca'] = cloud_Busan_outer['ca'].interpolate(method = 'values')
cloud_Busan_outer['ch'] = cloud_Busan_outer['ch'].interpolate(method = 'values')

cloud_Busan_outer['add'] = '부산'
del cloud_Busan_outer['stn_id']
del cloud_Busan_outer['lat']
del cloud_Busan_outer['long']
del cloud_Busan_outer['ch']


In [33]:
len(set(cloud_Busan_outer['date']))

3966

In [73]:
cloud_Busan_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 서울

In [34]:
cloud_Seoul_outer = pd.merge(cloud_Seoul, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Seoul_outer['ca'] = cloud_Seoul_outer['ca'].interpolate(method = 'values')
cloud_Seoul_outer['ch'] = cloud_Seoul_outer['ch'].interpolate(method = 'values')

cloud_Seoul_outer['add'] = '서울'
del cloud_Seoul_outer['stn_id']
del cloud_Seoul_outer['lat']
del cloud_Seoul_outer['long']
del cloud_Seoul_outer['ch']

In [35]:
len(set(cloud_Seoul_outer['date']))

3966

In [74]:
cloud_Seoul_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 세종

In [36]:
cloud_Sejong_outer = pd.merge(cloud_Sejong, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Sejong_outer['ca'] = cloud_Sejong_outer['ca'].interpolate(method = 'values')
cloud_Sejong_outer['ch'] = cloud_Sejong_outer['ch'].interpolate(method = 'values')

cloud_Sejong_outer['add'] = '세종'
del cloud_Sejong_outer['stn_id']
del cloud_Sejong_outer['lat']
del cloud_Sejong_outer['long']

In [38]:
len(set(cloud_Sejong_outer['date']))

3926

In [75]:
cloud_Sejong_outer.isna().sum()

date    0
ca      0
dtype: int64

In [78]:
Sejong_imp = pd.merge(cloud_Sejong_outer, days, on = ['date'], how = 'outer').sort_values(by = 'date')
for_Sejong = pd.concat([cloud_Daejeon_outer,
                        Sejong_imp,
                        cloud_Chungnam_outer,
                        cloud_Chungbuk_outer], axis=0)
cloud_Sejong_outer = for_Sejong[for_Sejong['add'].isin(['충남', '충북', '대전'])].groupby(['date'], as_index=False).mean().assign(add='세종')
# del cloud_Sejong_outer['ch']

In [48]:
len(set(cloud_Sejong_outer['date']))

3966

In [79]:
cloud_Sejong_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 울산

In [40]:
cloud_Ulsan_outer = pd.merge(cloud_Ulsan, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Ulsan_outer['ca'] = cloud_Ulsan_outer['ca'].interpolate(method = 'values')
cloud_Ulsan_outer['ch'] = cloud_Ulsan_outer['ch'].interpolate(method = 'values')

cloud_Ulsan_outer['add'] = '울산'
del cloud_Ulsan_outer['stn_id']
del cloud_Ulsan_outer['lat']
del cloud_Ulsan_outer['long']
del cloud_Ulsan_outer['ch']

In [49]:
len(set(cloud_Ulsan_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 인천

In [41]:
cloud_Incheon_outer = pd.merge(cloud_Incheon, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Incheon_outer['ca'] = cloud_Incheon_outer['ca'].interpolate(method = 'values')
cloud_Incheon_outer['ch'] = cloud_Incheon_outer['ch'].interpolate(method = 'values')

cloud_Incheon_outer['add'] = '인천'
del cloud_Incheon_outer['stn_id']
del cloud_Incheon_outer['lat']
del cloud_Incheon_outer['long']
del cloud_Incheon_outer['ch']

In [50]:
len(set(cloud_Incheon_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 전남

In [42]:
cloud_Jeonnam_outer = pd.merge(cloud_Jeonnam, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Jeonnam_outer['ca'] = cloud_Jeonnam_outer['ca'].interpolate(method = 'values')
cloud_Jeonnam_outer['ch'] = cloud_Jeonnam_outer['ch'].interpolate(method = 'values')

cloud_Jeonnam_outer['add'] = '전남'
del cloud_Jeonnam_outer['stn_id']
del cloud_Jeonnam_outer['lat']
del cloud_Jeonnam_outer['long']
del cloud_Jeonnam_outer['ch']

In [51]:
len(set(cloud_Jeonnam_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 전북

In [43]:
cloud_Jeonbuk_outer = pd.merge(cloud_Jeonbuk, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Jeonbuk_outer['ca'] = cloud_Jeonbuk_outer['ca'].interpolate(method = 'values')
cloud_Jeonbuk_outer['ch'] = cloud_Jeonbuk_outer['ch'].interpolate(method = 'values')

cloud_Jeonbuk_outer['add'] = '전북'
del cloud_Jeonbuk_outer['stn_id']
del cloud_Jeonbuk_outer['lat']
del cloud_Jeonbuk_outer['long']
del cloud_Jeonbuk_outer['ch']

In [52]:
len(set(cloud_Jeonbuk_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 제주

In [44]:
cloud_Jeju_outer = pd.merge(cloud_Jeju, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Jeju_outer['ca'] = cloud_Jeju_outer['ca'].interpolate(method = 'values')
cloud_Jeju_outer['ch'] = cloud_Jeju_outer['ch'].interpolate(method = 'values')

cloud_Jeju_outer['add'] = '제주'
del cloud_Jeju_outer['stn_id']
del cloud_Jeju_outer['lat']
del cloud_Jeju_outer['long']
del cloud_Jeju_outer['ch']

In [53]:
len(set(cloud_Jeju_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 충남

In [45]:
cloud_Chungnam_outer = pd.merge(cloud_Chungnam, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Chungnam_outer['ca'] = cloud_Chungnam_outer['ca'].interpolate(method = 'values')
cloud_Chungnam_outer['ch'] = cloud_Chungnam_outer['ch'].interpolate(method = 'values')

cloud_Chungnam_outer['add'] = '충남'
del cloud_Chungnam_outer['stn_id']
del cloud_Chungnam_outer['lat']
del cloud_Chungnam_outer['long']
del cloud_Chungnam_outer['ch']


In [54]:
len(set(cloud_Chungnam_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

## 충북 

In [46]:
cloud_Chungbuk_outer = pd.merge(cloud_Chungbuk, days, on = ['date'], how = 'outer').sort_values(by = 'date')

cloud_Chungbuk_outer['ca'] = cloud_Chungbuk_outer['ca'].interpolate(method = 'values')
cloud_Chungbuk_outer['ch'] = cloud_Chungbuk_outer['ch'].interpolate(method = 'values')

cloud_Chungbuk_outer['add'] = '충북'
del cloud_Chungbuk_outer['stn_id']
del cloud_Chungbuk_outer['lat']
del cloud_Chungbuk_outer['long']
del cloud_Chungbuk_outer['ch']

In [55]:
len(set(cloud_Chungbuk_outer['date']))

3966

In [66]:
cloud_Gangwon_outer.isna().sum()

date    0
ca      0
add     0
dtype: int64

In [80]:
cloud_full = pd.concat([cloud_Gangwon_outer,
           cloud_Gyeonggi_outer, 
           cloud_Gyeongbuk_outer, 
           cloud_Gyeongnam_outer,
           cloud_Gwangju_outer,
           cloud_Daegu_outer,
           cloud_Daejeon_outer,
           cloud_Busan_outer,
           cloud_Seoul_outer,
           cloud_Sejong_outer,
           cloud_Ulsan_outer,
           cloud_Incheon_outer,
           cloud_Jeonnam_outer,
           cloud_Jeonbuk_outer,
           cloud_Jeju_outer,
           cloud_Chungnam_outer,
           cloud_Chungbuk_outer], axis=0)
# 날짜별 - 시도별로 정렬되어 있어야 함

# cloud_full = cloud_full.rename(columns={'ca' : 'ca'})
cloud_full = cloud_full.groupby(['date', 'add'], as_index=False).mean()
cloud_full = cloud_full.sort_values(by = ['date','add'])
# 66442 행이 있어야 해

In [81]:
cloud_full.iloc[:67269]

,date,add,ca
0,2012-01-01,강원,5.953846
1,2012-01-01,경기,4.880000
2,2012-01-01,경남,4.075000
3,2012-01-01,경북,5.325000
4,2012-01-01,광주,8.300000
...,...,...,...
67264,2022-10-31,전남,2.784615
67265,2022-10-31,전북,3.620000
67266,2022-10-31,제주,3.325000
67267,2022-10-31,충남,4.471429


In [82]:
cloud_full.isna().sum()

date    0
add     0
ca      0
dtype: int64

In [83]:
cloud_full.iloc[:67269].to_csv('../data/cloud_for_merge.csv')

In [151]:
cloud_full

,date,add,ca
0,2012-01-01,강원,5.953846
1,2012-01-01,경기,4.880000
2,2012-01-01,경남,4.075000
3,2012-01-01,경북,5.325000
4,2012-01-01,광주,8.300000
...,...,...,...
63451,2022-11-09,전남,3.669231
63452,2022-11-09,전북,6.200000
63453,2022-11-09,제주,2.500000
63454,2022-11-09,충남,6.800000


In [10]:
pd.read_csv('../data/cloud_for_merge2.csv', index_col = 0)
#그냥 손으로 없애야 겠다

,date,add,ca
0,2012.1.1,媛__,5.953846
1,2012.1.1,寃쎄린,4.880000
2,2012.1.1,寃쎈_,4.075000
3,2012.1.1,寃쎈_,5.325000
4,2012.1.1,愿_＜,8.300000
...,...,...,...
63307,2022.10.31,__,2.784615
63308,2022.10.31,__,3.620000
63309,2022.10.31,_＜,3.325000
63310,2022.10.31,異⑸_,4.471429


# -------------------------------------

In [113]:
to_imp = pd.read_csv('../data/cloud_median.csv')
to_imp['date'] = pd.to_datetime(to_imp['date'])
to_imp['date'] = pd.to_datetime(to_imp['date'], format='%Y-%m-%d')

In [118]:
to_imp

,Unnamed: 0,date,add,stn_id,cloud.ca,cloud.ch,lat,long
0,0,2012-02-01,강원,114.0,7.000000,10.000000,37.337495,128.457415
1,1,2012-02-01,경기,161.0,3.500000,10.000000,37.260728,127.233603
2,2,2012-02-01,경남,192.0,6.000000,10.000000,35.170195,128.040020
3,3,2012-02-01,경북,115.0,6.000000,8.000000,37.481133,130.898795
4,4,2012-02-01,광주,156.0,6.000000,10.000000,35.172920,126.891580
...,...,...,...,...,...,...,...,...
54247,3238,2022-10-26,세종,147.0,7.500000,60.000000,36.556252,127.166832
54248,3239,2022-10-27,세종,147.0,5.666667,57.666667,36.556252,127.166832
54249,3240,2022-10-28,세종,147.0,9.000000,2.500000,36.556252,127.166832
54250,3241,2022-10-29,세종,147.0,3.666667,26.666667,36.556252,127.166832


In [112]:
cloud_full[cloud_full['ch'].isna()]

,date,add,ca,ch
0,2012-01-01,강원,5.953846,NaN
1,2012-01-01,경기,4.880000,NaN
2,2012-01-01,경남,4.075000,NaN
3,2012-01-01,경북,5.325000,NaN
4,2012-01-01,광주,8.300000,NaN
...,...,...,...,...
3066,2012-07-10,인천,9.283333,NaN
3082,2012-07-11,인천,9.000000,NaN
3098,2012-07-12,인천,9.233333,NaN
3114,2012-07-13,인천,8.966667,NaN


In [116]:
imp = pd.merge(cloud_full[cloud_full['ch'].isna()], to_imp, on = ['date', 'add']).sort_values(by = 'date')
del imp['cloud.ca']
del imp['ch']
del imp['Unnamed: 0']
del imp['lat']
del imp['long']

imp = imp.rename(columns={'cloud.ch' : 'ch'})

In [117]:
imp

,date,add,ca,stn_id,ch
0,2012-02-01,경기,0.828000,161.0,10.0
1,2012-02-01,대구,1.600000,143.0,10.0
2,2012-02-01,대전,2.600000,133.0,12.0
3,2012-02-01,부산,1.400000,159.0,10.0
4,2012-02-01,서울,0.000000,108.0,10.0
...,...,...,...,...,...
395,2012-07-10,인천,9.283333,102.0,24.0
396,2012-07-11,인천,9.000000,112.0,10.0
397,2012-07-12,인천,9.233333,112.0,10.0
398,2012-07-13,인천,8.966667,112.0,10.0


In [ ]:
cloud_full[cloud_full['ch'].isna()]